In [3]:
import tensorflow
from util import PATH_FOR_OUR_TRAINING_DATA, UPPER_LIMIT_OF_IMAGES, TARGET_SHAPE
import pandas as pd
from DS_Generator import DS_Generator, smart_batches
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from util import PATH_FOR_OUR_TRAINING_DATA

In [5]:
df = pd.read_csv(PATH_FOR_OUR_TRAINING_DATA + "/data.csv", index_col=0)
df = df[df["species_counts"]>100].sort_values(["species_counts"])
ds = DS_Generator().generate(df)

In [2]:
df = pd.read_csv(PATH_FOR_OUR_TRAINING_DATA + "/data.csv", index_col=0)

In [7]:
smart_batches(df,64,"species")

We threw away the datapoint with index 13883 


,image,species,individual_id,individum_count,label,species_label,species_counts,assign_to
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,1,12348,18,1689,0.0
42088,d360a7a18c49a8.jpg,pantropic_spotted_dolphin,a8bc24429742,2,10306,20,145,0.0
45973,e753476f4ea858.jpg,beluga,a0d1a0e2e606,30,9820,0,7443,0.0
44847,e1685c78a5d52c.jpg,humpback_whale,35f898e6595e,57,3265,14,7392,0.0
41934,d29a003ed27578.jpg,spotted_dolphin,a0702f21bf13,1,9798,28,490,0.0
...,...,...,...,...,...,...,...,...
48466,f33a8189f3e5b9.jpg,killer_whale,e9433701eade,5,14227,16,1493,797.0
10656,35a621ba67802e.jpg,beluga,122dc7f46c24,30,1136,0,7443,797.0
42851,d738428b2d9cbb.jpg,pantropic_spotted_dolphin,28cca2cd7019,1,2522,20,145,797.0
39844,c86272f5f54146.jpg,blue_whale,ed72710d3872,4,14485,1,4830,797.0


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=TARGET_SHAPE+(3,)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [7]:
model.fit(
    ds,
    epochs=5)

Epoch 1/5
393/393 [==============================] - 80s 196ms/step - loss: 1.0054
Epoch 2/5
393/393 [==============================] - 77s 197ms/step - loss: 1.0020
Epoch 3/5
393/393 [==============================] - 80s 202ms/step - loss: 1.0018
Epoch 4/5
393/393 [==============================] - 81s 206ms/step - loss: 1.0017
Epoch 5/5
393/393 [==============================] - 79s 202ms/step - loss: 1.0016
